In [434]:
import os
import pandas as pd

# Set the directory where the text files are located
songs_dir = "songs"
data = []
# Loop through each file in the directory
for root, dirs, files in os.walk(songs_dir):
    for file in files:
        if file.endswith(".txt"):
            artist = os.path.basename(root)
            with open(os.path.join(root, file), 'r', encoding="utf8") as f:
                lyrics = f.read().replace('\n', ' ')
                # Add the data to the DataFrame
                data.append([artist, lyrics])
                
# Create an empty DataFrame to store the data
df = pd.DataFrame(data, columns=['Artist', 'Lyrics'])
# Export the DataFrame to a CSV file
df.to_csv('lyrics.csv', index=False)


In [435]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Load the data from the CSV file
data = pd.read_csv("lyrics.csv")

nltk.download('stopwords')
nltk.download('punkt')

# Create a list of stopwords to remove
stop_words = set(stopwords.words("english"))
stop_words.add("verse")
stop_words.add("intro")

# Create a stemmer to use for word stemming
stemmer = PorterStemmer()

# Preprocess each lyric in the DataFrame
for i, row in data.iterrows():
#Convert the lyric to lowercase
    lyric = str(row["Lyrics"]).lower()
    match = re.search(r'lyrics\[[^\]]*\]', lyric)

    # Check if the split was successful
    if match:
        split_index = match.end()
        cleaned_lyric = lyric[split_index:]
    else:
        cleaned_lyric = lyric

    #Tokenize the lyric into words
    words = word_tokenize(cleaned_lyric)

    #Remove stop words and punctuation
    filtered_words = [word for word in words if word.isalpha() and word not in stop_words]

    #Stem each word
    stemmed_words = [stemmer.stem(word) for word in filtered_words]

    #Join the stemmed words back into a single string
    preprocessed_lyric = " ".join(stemmed_words)

    #Replace the original lyric with the preprocessed lyric in the DataFrame
    data.at[i, "Lyrics"] = preprocessed_lyric

# Export the preprocessed DataFrame to a CSV file
data.to_csv("preprocessed_lyrics.csv", index=False)



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jarraomar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jarraomar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [436]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

pre_processed_data = pd.read_csv("preprocessed_lyrics.csv")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pre_processed_data["Lyrics"])
sequences = tokenizer.texts_to_sequences(pre_processed_data["Lyrics"])

In [438]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#For Artist Classification

artists = pre_processed_data["Artist"].unique()
# Encode the artist names as integer labels
label_encoder = LabelEncoder()
pre_processed_data["Artist"] = label_encoder.fit_transform(pre_processed_data["Artist"])

# Initialize empty dataframes for training, validation, and testing
train_df = pd.DataFrame(columns=["Artist", "Lyrics"])
val_df = pd.DataFrame(columns=["Artist", "Lyrics"])
test_df = pd.DataFrame(columns=["Artist", "Lyrics"])

for artist in artists:
    # Get the data for the current artist
    artist_data = pre_processed_data[pre_processed_data["Artist"] == artist]
    
    # Split the artist data into training, validation, and testing sets
    artist_train, artist_test = train_test_split(artist_data, test_size=0.2, random_state=42)
    artist_train, artist_val = train_test_split(artist_train, test_size=0.2, random_state=42)
    
    # Concatenate the artist training, validation, and testing dataframes with the overall training, validation, and testing dataframes
    train_df = pd.concat([train_df, artist_train])
    val_df = pd.concat([val_df, artist_val])
    test_df = pd.concat([test_df, artist_test])

# Create a directory to store the CSV files
directory = "data_splits"
if not os.path.exists(directory):
    os.makedirs(directory)
    train_df.to_csv(os.path.join(directory, "train.csv"), index=False)
    val_df.to_csv(os.path.join(directory, "val.csv"), index=False)
    test_df.to_csv(os.path.join(directory, "test.csv"), index=False)

train_df.to_csv(os.path.join(directory, "train.csv"), index=False)
val_df.to_csv(os.path.join(directory, "val.csv"), index=False)
test_df.to_csv(os.path.join(directory, "test.csv"), index=False)


In [439]:
import pandas as pd
import numpy as np
import gensim
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

train = pd.read_csv('data_splits/train.csv')
val = pd.read_csv('data_splits/val.csv')
test = pd.read_csv('data_splits/test.csv')

#Creation of MLP model:
model_mlp = Sequential()
model_mlp.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=300, input_length=200))
model_mlp.add(GlobalMaxPooling1D())
model_mlp.add(Dense(100, activation='relu'))
model_mlp.add(Dropout(0.4))
model_mlp.add(Dense(128, activation='relu'))
model_mlp.add(Dropout(0.5))
model_mlp.add(Dense(128, activation='relu'))
model_mlp.add(Dropout(0.5))
model_mlp.add(Dense(8, activation='softmax'))

tokenizer.fit_on_texts(train['Lyrics'])
sequences_train = tokenizer.texts_to_sequences(train['Lyrics'])
sequences_val = tokenizer.texts_to_sequences(val['Lyrics'])
sequences_test = tokenizer.texts_to_sequences(test['Lyrics'])
word_index = tokenizer.word_index
X_train = pad_sequences(sequences_train, maxlen=200)
X_val = pad_sequences(sequences_val, maxlen=200)
X_test = pad_sequences(sequences_test, maxlen=200)
    
# One-hot encode the target variable
y_train = to_categorical(train['Artist'])
y_val = to_categorical(val['Artist'])
y_test = to_categorical(test['Artist'])


In [454]:
model_mlp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 300)          2429400   
                                                                 
 global_max_pooling1d (Globa  (None, 300)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 100)               30100     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               12928     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

In [440]:
#We apply earlystopping in order to avoid over-fitting
es = EarlyStopping(monitor='val_loss', patience=5)
model_mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [443]:
model_mlp.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=64, callbacks=[es])

Epoch 1/100
4/4 [==============================] - 0s 98ms/step - loss: 0.1629 - accuracy: 0.9609 - val_loss: 0.7793 - val_accuracy: 0.7656
Epoch 2/100
4/4 [==============================] - 0s 70ms/step - loss: 0.1476 - accuracy: 0.9531 - val_loss: 0.7963 - val_accuracy: 0.7344
Epoch 3/100
4/4 [==============================] - 0s 67ms/step - loss: 0.1617 - accuracy: 0.9570 - val_loss: 0.8465 - val_accuracy: 0.7188
Epoch 4/100
4/4 [==============================] - 0s 64ms/step - loss: 0.1625 - accuracy: 0.9492 - val_loss: 0.8285 - val_accuracy: 0.7344
Epoch 5/100
4/4 [==============================] - 0s 48ms/step - loss: 0.1256 - accuracy: 0.9570 - val_loss: 0.8281 - val_accuracy: 0.7656
Epoch 6/100
4/4 [==============================] - 0s 48ms/step - loss: 0.1512 - accuracy: 0.9531 - val_loss: 0.8376 - val_accuracy: 0.7344


In [444]:
#Testing our model's accuracy based on a separate test set. "X and Y test"
test_loss, test_acc = model_mlp.evaluate(X_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

3/3 - 0s - loss: 0.9137 - accuracy: 0.7250 - 31ms/epoch - 10ms/step
Test accuracy: 0.7250000238418579


In [445]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.models import load_model

model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=len(tokenizer.word_index) + 1, input_length=200, output_dim=10))
model_lstm.add(Bidirectional(LSTM(15, return_sequences=True)))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(10)) 
model_lstm.add(Dense(10, activation='relu'))
model_lstm.add(Dense(20, activation='softmax'))

model_lstm

2023-05-18 13:53:52.001302: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-18 13:53:52.003965: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-18 13:53:52.006467: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [453]:
model_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 10)           80980     
                                                                 
 bidirectional (Bidirectiona  (None, 200, 30)          3120      
 l)                                                              
                                                                 
 dropout_3 (Dropout)         (None, 200, 30)           0         
                                                                 
 lstm_1 (LSTM)               (None, 10)                1640      
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 20)                220       
                                                      

In [446]:
generated_lyrics_dir = "Generated Lyrics"
if not os.path.exists(generated_lyrics_dir):
    os.makedirs(generated_lyrics_dir)

generated_lyrics_list = []

# Set the maximum number of words to generate for each artist
max_words = max_lines * max_line_length

# Iterate through each artist
for artist in artists:
    artist_str = str(artist)

    # Generate 50 songs for the current artist
    for _ in range(50):
        # Filter the dataset for the current artist
        artist_data = pre_processed_data[pre_processed_data["Artist"] == artist]

        artist_words = artist_data["Lyrics"].str.split().sum()

        # Shuffle the list of words
        random.shuffle(artist_words)

        # Generate lyrics line by line
        lines_generated = 0
        current_line = ""
        generated_lyrics = ""
        generated_lyrics_csv = ""

        # Create the seed by joining the shuffled artist words
        seed = ' '.join(artist_words)
        words_generated = 0

        while words_generated < 200:

            # Get the next word from the shuffled list
            if len(current_line.split()) % 3 == 0:
                next_words = 1
                token_list = tokenizer.texts_to_sequences([seed])[0]
                token_list = pad_sequences([token_list], maxlen=200, padding='pre')
                predicted_probs = model_lstm.predict(token_list)[0]
                predicted_index = np.random.choice(range(len(predicted_probs)), size=next_words, p=predicted_probs)[0]
                next_word = tokenizer.index_word.get(predicted_index, '')
                words_generated += 1
            
            else:
                next_word = artist_words.pop(0)
                words_generated += 1

            # Truncate the word if it exceeds the maximum line length
            if len(next_word) > max_line_length:
                next_word = next_word[:max_line_length]

            current_line += next_word + " "

        # Append the last line to the generated lyrics
        generated_lyrics_csv += current_line.strip()

        generated_lyrics_list.append({"Artist": artist, "Lyrics": generated_lyrics_csv})

csv_file = os.path.join(generated_lyrics_dir, "gen_lyrics.csv")
df_gen_lyrics = pd.DataFrame(columns=["Artist", "Lyrics"])
df_gen_lyrics = df_gen_lyrics.append(generated_lyrics_list, ignore_index=True)
df_gen_lyrics.to_csv(csv_file, index=False)

2023-05-18 13:53:59.893000: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-18 13:53:59.894755: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-18 13:53:59.896268: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 37ms/step


/var/folders/_5/5yx0jmr50jzfllwklwqq4mh00000gn/T/ipykernel_85677/1069912174.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_gen_lyrics = df_gen_lyrics.append(generated_lyrics_list, ignore_index=True)


In [452]:
gen_lyrics = pd.read_csv('Generated Lyrics/gen_lyrics.csv')
X_gen = gen_lyrics["Lyrics"]
X_gen_sequences = tokenizer.texts_to_sequences(X_gen)

# Pad the sequences to have consistent length
X_gen_padded = pad_sequences(X_gen_sequences, maxlen=200)

# Make predictions on the generated lyrics
predictions = model_mlp.predict(X_gen_padded)

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the ground truth labels for the generated lyrics
ground_truth_labels = label_encoder.transform(gen_lyrics["Artist"])

# Calculate accuracy
accuracy = np.mean(predicted_labels == ground_truth_labels)

print('Test accuracy:', accuracy)

13/13 [==============================] - 0s 4ms/step
Test accuracy: 0.9525


In [448]:
generated_lyrics_file = "Generated Lyrics/gen_lyrics.csv"
completed_songs_dir = "restructured_lyrics"

# Read the generated lyrics CSV file
df = pd.read_csv(generated_lyrics_file)

# Create the completed songs directory if it doesn't exist
if not os.path.exists(completed_songs_dir):
    os.makedirs(completed_songs_dir)

# Iterate through each artist in the generated lyrics
for artist in df['Artist'].unique():
    artist_dir = os.path.join(completed_songs_dir, str(artist))
    
    # Create a new directory for the completed songs for the current artist
    if not os.path.exists(artist_dir):
        os.makedirs(artist_dir)
    
    # Get the generated lyrics for the current artist
    artist_lyrics = df.loc[df['Artist'] == artist, 'Lyrics'].values
    
    # Iterate through each song of the current artist
    for i, lyrics in enumerate(artist_lyrics):
        # Create song lines based on maximum line length
        lines_generated = 0
        current_line = ""
        song_lines = []
        
        words = lyrics.split()
        for word in words:
            if len(current_line.split()) >= max_line_length:
                song_lines.append(current_line.strip())
                current_line = ""
                lines_generated += 1
            
            if len(word) > max_line_length:
                word = word[:max_line_length]
            
            current_line += word + " "
        
        if current_line.strip():
            song_lines.append(current_line.strip())
        
        # Write the original song lines to the completed song file
        song_file = f"song_{i+1}.txt"
        completed_song_path = os.path.join(artist_dir, song_file)
        
        with open(completed_song_path, "w") as f:
            f.write("\n".join(song_lines))


In [433]:
from keras import backend as K
K.clear_session()
del model_mlp
del model_lstm